# Correct data for specific channels

## Correction for diferent ADC Values and 5 runs of each

In [56]:
import pandas as pd
import os

ADC_values = [100, 200, 300, 400]
RUNS = range(1, 6)
channels_to_replace = list(range(133, 151))

for ADC in ADC_values:
    for RUN in RUNS:
        print(f"\n🔄 Processing ADC = {ADC}, RUN = {RUN}...")

        # Define file paths
        file_main = f'./internalinjection/{ADC}dac/inj_adc_samples_209_pb008_76I_{ADC}_{RUN}.csv'
        file_source = f'./internalinjection/{ADC}dac/inj_adc_samples_209_pb008_19_{ADC}_{RUN}.csv'

        # Check if both files exist
        if not os.path.exists(file_main):
            print(f"⚠️ File not found: {file_main}")
            continue
        if not os.path.exists(file_source):
            print(f"⚠️ File not found: {file_source}")
            continue

        # Load CSV files
        df_main = pd.read_csv(file_main)
        df_source = pd.read_csv(file_source)

        # Prepare replacement rows
        filtered_replacements = []
        for ch in channels_to_replace:
            main_ch = df_main[df_main['Channel'] == ch]
            src_ch = df_source[df_source['Channel'] == ch]

            # Keep only matching phases
            matched = src_ch[src_ch['Phase'].isin(main_ch['Phase'])]
            filtered_replacements.append(matched)

        # Combine updated channel rows
        replacement_block = pd.concat(filtered_replacements, ignore_index=True)

        # Remove old channels 77–94 from main file
        df_main_wo_ch = df_main[~df_main['Channel'].isin(channels_to_replace)].reset_index(drop=True)

        # Find insertion index (first where Channel > 132)
        insert_locs = df_main_wo_ch[df_main_wo_ch['Channel'] > 132].index
    
        if len(insert_locs) == 0:
            # No higher channels found, insert at end
            insert_index = len(df_main_wo_ch)
        else:
            insert_index = insert_locs.min()
        
        # Insert new data
        before = df_main_wo_ch.iloc[:insert_index]
        after = df_main_wo_ch.iloc[insert_index:]
        df_final = pd.concat([before, replacement_block, after], ignore_index=True)

        # Save output
        output_path = f'./internalinjection/{ADC}dac/inj_adc_samples_209_pb008_76_{ADC}_{RUN}.csv'
        df_final.to_csv(output_path, index=False)

        print(f"✅ Done: {output_path}")



🔄 Processing ADC = 100, RUN = 1...
✅ Done: ./internalinjection/100dac/inj_adc_samples_209_pb008_76_100_1.csv

🔄 Processing ADC = 100, RUN = 2...
✅ Done: ./internalinjection/100dac/inj_adc_samples_209_pb008_76_100_2.csv

🔄 Processing ADC = 100, RUN = 3...
✅ Done: ./internalinjection/100dac/inj_adc_samples_209_pb008_76_100_3.csv

🔄 Processing ADC = 100, RUN = 4...
✅ Done: ./internalinjection/100dac/inj_adc_samples_209_pb008_76_100_4.csv

🔄 Processing ADC = 100, RUN = 5...
✅ Done: ./internalinjection/100dac/inj_adc_samples_209_pb008_76_100_5.csv

🔄 Processing ADC = 200, RUN = 1...
✅ Done: ./internalinjection/200dac/inj_adc_samples_209_pb008_76_200_1.csv

🔄 Processing ADC = 200, RUN = 2...
✅ Done: ./internalinjection/200dac/inj_adc_samples_209_pb008_76_200_2.csv

🔄 Processing ADC = 200, RUN = 3...
✅ Done: ./internalinjection/200dac/inj_adc_samples_209_pb008_76_200_3.csv

🔄 Processing ADC = 200, RUN = 4...
✅ Done: ./internalinjection/200dac/inj_adc_samples_209_pb008_76_200_4.csv

🔄 Process

## Replace individual channel

In [ ]:
import pandas as pd
import os

ADC_values = [100, 200, 300, 400]
RUNS = range(1, 6)
channels_to_replace = 99

for ADC in ADC_values:
    for RUN in RUNS:
        print(f"\n🔄 Processing ADC = {ADC}, RUN = {RUN}...")

        # Define file paths
        file_main = f'./internalinjection/{ADC}dac/inj_adc_samples_209_pb008_76I_{ADC}_{RUN}.csv'
        file_source = f'./internalinjection/{ADC}dac/inj_adc_samples_209_pb008_19_{ADC}_{RUN}.csv'

        # Check if both files exist
        if not os.path.exists(file_main):
            print(f"⚠️ File not found: {file_main}")
            continue
        if not os.path.exists(file_source):
            print(f"⚠️ File not found: {file_source}")
            continue

        # Load CSV files
        df_main = pd.read_csv(file_main)
        df_source = pd.read_csv(file_source)

        # Prepare replacement rows
        filtered_replacements = []
        for ch in channels_to_replace:
            main_ch = df_main[df_main['Channel'] == ch]
            src_ch = df_source[df_source['Channel'] == ch]

            # Keep only matching phases
            matched = src_ch[src_ch['Phase'].isin(main_ch['Phase'])]
            filtered_replacements.append(matched)

        # Combine updated channel rows
        replacement_block = pd.concat(filtered_replacements, ignore_index=True)

        # Remove old channels 77–94 from main file
        df_main_wo_ch = df_main[~df_main['Channel'].isin(channels_to_replace)].reset_index(drop=True)

        # Find insertion index (first where Channel > 132)
        insert_locs = df_main_wo_ch[df_main_wo_ch['Channel'] > 132].index
    
        if len(insert_locs) == 0:
            # No higher channels found, insert at end
            insert_index = len(df_main_wo_ch)
        else:
            insert_index = insert_locs.min()
        
        # Insert new data
        before = df_main_wo_ch.iloc[:insert_index]
        after = df_main_wo_ch.iloc[insert_index:]
        df_final = pd.concat([before, replacement_block, after], ignore_index=True)

        # Save output
        output_path = f'./internalinjection/{ADC}dac/inj_adc_samples_209_pb008_76_{ADC}_{RUN}.csv'
        df_final.to_csv(output_path, index=False)

        print(f"✅ Done: {output_path}")


## Correction of channel even if there are not channel data in file to be corrected

1. The output file is always generated if the main file exists.

2. The replacement step is skipped if the source file is missing or if it does not contain the channels to replace. Uncomment 
```
if not os.path.exists(file_source):
    print(f"⚠️ File not found: {file_source}")
    continue
```

3. A message clarifies whether a replacement was done or skipped.

In [ ]:
import pandas as pd
import os

ADC_values = [400]
RUNS = [5]
channels_to_replace = list(range(39, 74))  # Use a list so you can loop even if it's one channel

for ADC in ADC_values:
    for RUN in RUNS:
        print(f"\n🔄 Processing ADC = {ADC}, RUN = {RUN}...")

        # Define file paths
        file_main = f'./internalinjection/TOT_Test/{ADC}dac/inj_adc_samples_208_pb01_76I_{ADC}_{RUN}.csv'
        file_source = f'./internalinjection/TOT_Test/{ADC}dac/inj_adc_samples_208_pb01_36_{ADC}_{RUN}.csv'

        # Check if both files exist
        if not os.path.exists(file_main):
            print(f"⚠️ File not found: {file_main}")
            continue
        if not os.path.exists(file_source):
            print(f"⚠️ File not found: {file_source}")
            continue

        # Load CSV files
        df_main = pd.read_csv(file_main)
        df_source = pd.read_csv(file_source)

        # Ensure columns are stripped of whitespace
        df_main.columns = df_main.columns.str.strip()
        df_source.columns = df_source.columns.str.strip()

        # Remove the old channels (if they exist) from df_main
        df_main = df_main[~df_main['Channel'].isin(channels_to_replace)]

        # Extract new rows from source for the given channels
        new_rows = df_source[df_source['Channel'].isin(channels_to_replace)]

        if new_rows.empty:
            print(f"⚠️ No replacement data found in source file for channels {channels_to_replace}")
            continue

        # Combine and sort by Channel and Phase
        df_final = pd.concat([df_main, new_rows], ignore_index=True)
        df_final = df_final.sort_values(by=['Channel', 'Phase']).reset_index(drop=True)

        # Save output
        output_path = f'./internalinjection/{ADC}dac/inj_adc_samples_209_pb008_76_{ADC}_{RUN}.csv'
        df_final.to_csv(output_path, index=False)

        print(f"✅ Done: {output_path}")


#### For ToT

In [4]:
import pandas as pd
import os

ADC_values = [400, 925, 1450]
RUNS = [2, 3, 4, 5]
channels_to_replace = list(range(39, 75))  # Use a list so you can loop even if it's one channel

for ADC in ADC_values:
    for RUN in RUNS:
        print(f"\n🔄 Processing ADC = {ADC}, RUN = {RUN}...")

        # Define file paths
        file_main = f'./internalinjection/TOT_Test/{ADC}dac/inj_adc_samples_208_pb01_76I_{ADC}_{RUN}.csv'
        file_source = f'./internalinjection/TOT_Test/{ADC}dac/inj_adc_samples_208_pb01_36_{ADC}_{RUN}.csv'
        output_path = f'./internalinjection/TOT_Test/{ADC}dac/inj_adc_samples_208_pb01_76_{ADC}_{RUN}.csv'

        # Check if both files exist
        if not os.path.exists(file_main):
            print(f"⚠️ File not found: {file_main}")
            continue
        # if not os.path.exists(file_source):
        #     print(f"⚠️ File not found: {file_source}")
        #     continue

        # Load CSV files
        df_main = pd.read_csv(file_main)
        # Ensure columns are stripped of whitespace
        df_main.columns = df_main.columns.str.strip()

        # Initialize final dataframe with just the main file
        df_final = df_main.copy()

        # Try replacing channels only if the source file exists
        if os.path.exists(file_source):
            df_source = pd.read_csv(file_source)
            df_source.columns = df_source.columns.str.strip()
            
            # Extract new rows from source for the given channels
            new_rows = df_source[df_source['Channel'].isin(channels_to_replace)]

            if not new_rows.empty:
                # Remove the old channels (if they exist) from df_main
                df_main = df_main[~df_main['Channel'].isin(channels_to_replace)]
                # Append new rows
                df_final = pd.concat([df_final, new_rows], ignore_index=True)
                print(f"🔁 Replaced channels {channels_to_replace} from source file.")
            else:
                print(f"⚠️ Source file found but no matching channels in {channels_to_replace}")
                # continue
        else:
            print(f"⚠️ Source file not found: {file_source}. No replacement done.")


        # Combine and sort by Channel and Phase
        # df_final = pd.concat([df_main, new_rows], ignore_index=True)
        df_final = df_final.sort_values(by=['Channel', 'Phase']).reset_index(drop=True)

        # Save output
        df_final.to_csv(output_path, index=False)

        print(f"✅ Done: {output_path}")



🔄 Processing ADC = 400, RUN = 2...
⚠️ File not found: ./internalinjection/TOT_Test/400dac/inj_adc_samples_208_pb01_76I_400_2.csv

🔄 Processing ADC = 400, RUN = 3...
⚠️ File not found: ./internalinjection/TOT_Test/400dac/inj_adc_samples_208_pb01_76I_400_3.csv

🔄 Processing ADC = 400, RUN = 4...
⚠️ File not found: ./internalinjection/TOT_Test/400dac/inj_adc_samples_208_pb01_76I_400_4.csv

🔄 Processing ADC = 400, RUN = 5...
🔁 Replaced channels [39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74] from source file.
✅ Done: ./internalinjection/TOT_Test/400dac/inj_adc_samples_208_pb01_76_400_5.csv

🔄 Processing ADC = 925, RUN = 2...
🔁 Replaced channels [39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74] from source file.
✅ Done: ./internalinjection/TOT_Test/925dac/inj_adc_samples_208_pb01_76_925_2.csv

🔄 Proce

In [5]:
import pandas as pd
import os

ADC_values = [1450, 1975]
RUNS = [2, 3, 4, 5]
channels_to_replace = list(range(77, 113))  # Use a list so you can loop even if it's one channel

for ADC in ADC_values:
    for RUN in RUNS:
        print(f"\n🔄 Processing ADC = {ADC}, RUN = {RUN}...")

        # Define file paths
        file_main = f'./internalinjection/TOT_Test/{ADC}dac/inj_adc_samples_208_pb01_76I_{ADC}_{RUN}.csv'
        file_source = f'./internalinjection/TOT_Test/{ADC}dac/inj_adc_samples_208_pb01_36_{ADC}_{RUN}.csv'
        output_path = f'./internalinjection/TOT_Test/{ADC}dac/inj_adc_samples_208_pb01_76_{ADC}_{RUN}.csv'

        # Check if both files exist
        if not os.path.exists(file_main):
            print(f"⚠️ File not found: {file_main}")
            continue
        # if not os.path.exists(file_source):
        #     print(f"⚠️ File not found: {file_source}")
        #     continue

        # Load CSV files
        df_main = pd.read_csv(file_main)
        # Ensure columns are stripped of whitespace
        df_main.columns = df_main.columns.str.strip()

        # Initialize final dataframe with just the main file
        df_final = df_main.copy()

        # Try replacing channels only if the source file exists
        if os.path.exists(file_source):
            df_source = pd.read_csv(file_source)
            df_source.columns = df_source.columns.str.strip()
            
            # Extract new rows from source for the given channels
            new_rows = df_source[df_source['Channel'].isin(channels_to_replace)]

            if not new_rows.empty:
                # Remove the old channels (if they exist) from df_main
                df_main = df_main[~df_main['Channel'].isin(channels_to_replace)]
                # Append new rows
                df_final = pd.concat([df_final, new_rows], ignore_index=True)
                print(f"🔁 Replaced channels {channels_to_replace} from source file.")
            else:
                print(f"⚠️ Source file found but no matching channels in {channels_to_replace}")
                # continue
        else:
            print(f"⚠️ Source file not found: {file_source}. No replacement done.")


        # Combine and sort by Channel and Phase
        # df_final = pd.concat([df_main, new_rows], ignore_index=True)
        df_final = df_final.sort_values(by=['Channel', 'Phase']).reset_index(drop=True)

        # Save output
        df_final.to_csv(output_path, index=False)

        print(f"✅ Done: {output_path}")



🔄 Processing ADC = 1450, RUN = 2...
⚠️ Source file found but no matching channels in [77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112]
✅ Done: ./internalinjection/TOT_Test/1450dac/inj_adc_samples_208_pb01_76_1450_2.csv

🔄 Processing ADC = 1450, RUN = 3...
⚠️ Source file found but no matching channels in [77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112]
✅ Done: ./internalinjection/TOT_Test/1450dac/inj_adc_samples_208_pb01_76_1450_3.csv

🔄 Processing ADC = 1450, RUN = 4...
🔁 Replaced channels [77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112] from source file.
✅ Done: ./internalinjection/TOT_Test/1450dac/inj_adc_samples_208_pb01_76_1450_4.csv

🔄 Processing ADC = 1450, R

## Correction of data for channels with and without HV

In [54]:
import pandas as pd
import os

ADC = 150
RUNS = ['Yes', 'HV']
channels_to_replace = list(range(133, 151))

base_dir = './internalinjection/SiPMs_not_connected_test/FPGA_209_pb008'

for run_tag in RUNS:
    print(f"\n🔄 Processing ADC = {ADC}, RUN = {run_tag}...")

    # Define file paths
    file_main = f'{base_dir}/inj_adc_samples_209_pb008_76I_{ADC}_{run_tag}.csv'
    file_source = f'{base_dir}/inj_adc_samples_209_pb008_19_{ADC}_{run_tag}.csv'

    # Check if both files exist
    if not os.path.exists(file_main):
        print(f"⚠️ File not found: {file_main}")
        continue
    if not os.path.exists(file_source):
        print(f"⚠️ File not found: {file_source}")
        continue

    # Load CSV files
    df_main = pd.read_csv(file_main)
    df_source = pd.read_csv(file_source)

    # Prepare replacement rows
    filtered_replacements = []
    for ch in channels_to_replace:
        main_ch = df_main[df_main['Channel'] == ch]
        src_ch = df_source[df_source['Channel'] == ch]

        matched = src_ch[src_ch['Phase'].isin(main_ch['Phase'])]
        filtered_replacements.append(matched)

    # Combine updated channel rows
    replacement_block = pd.concat(filtered_replacements, ignore_index=True)

    # Remove old channels from main
    df_main_wo_ch = df_main[~df_main['Channel'].isin(channels_to_replace)].reset_index(drop=True)

    # Find insertion index (first where Channel > 132)
    insert_locs = df_main_wo_ch[df_main_wo_ch['Channel'] > 132].index

    if len(insert_locs) == 0:
        # No higher channels found, insert at end
        insert_index = len(df_main_wo_ch)
    else:
        insert_index = insert_locs.min()
    
    # Insert new data
    before = df_main_wo_ch.iloc[:insert_index]
    after = df_main_wo_ch.iloc[insert_index:]
    df_final = pd.concat([before, replacement_block, after], ignore_index=True)

    # Save output
    output_file = f'{base_dir}/inj_adc_samples_209_pb008_76_{ADC}_{run_tag}.csv'
    df_final.to_csv(output_file, index=False)

    print(f"✅ Channels 133–150 updated for RUN = {run_tag}. Saved to: {output_file}")



🔄 Processing ADC = 150, RUN = Yes...
✅ Channels 133–150 updated for RUN = Yes. Saved to: ./internalinjection/SiPMs_not_connected_test/FPGA_209_pb008/inj_adc_samples_209_pb008_76_150_Yes.csv

🔄 Processing ADC = 150, RUN = HV...
✅ Channels 133–150 updated for RUN = HV. Saved to: ./internalinjection/SiPMs_not_connected_test/FPGA_209_pb008/inj_adc_samples_209_pb008_76_150_HV.csv
